In [ ]:
#!/usr/bin/python3
# coding: utf-8

In [ ]:
from datetime import datetime as dt
import sys
import numpy as np
import os
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as offline
import sys
if "ipy" in sys.argv[0]:
    offline.init_notebook_mode()
from cov19utils import create_basic_plot_figure, \
    show_and_clear, moving_average, \
    blank2zero, csv2array, \
    get_twitter, tweet_with_image, \
    get_gpr_predict, FONT_NAME, DT_OFFSET, \
    download_if_needed, json2nparr, code2int, age2int, \
    get_populations, get_os_idx_of_arr, dump_val_in_arr, \
    calc_last1w2w_dif, create_basic_scatter_figure, \
    show_and_save_plotly

In [ ]:
today_str = dt.now().isoformat()[:16].replace('T', ' ')
update_map = False

In [ ]:
with open("csv/longcovid.tmp", "rt") as f:
    prev = int(f.read())
imgname = "longcovid-map.jpg"
tw_body = '新型コロナ 後遺症外来マップ (' + today_str + ')'
df=pd.read_csv("csv/longcovid.csv", encoding='shift-jis', header=0)

In [ ]:
if len(df) > prev:
    update_map = True
    with open("csv/longcovid.tmp", "wt") as f:
        f.write("{}".format(len(df)))

In [ ]:
#update_map = True # TODO: AAWA

In [ ]:
fig = px.scatter_mapbox(df,
    title=tw_body,
    lat="lat", lon="lon", color="可", size="可",
    color_continuous_scale=plotly.colors.sequential.Bluered,
    hover_name="施設名称", 
    hover_data=["オンライン", "紹介状", "条件"],
    labels={'lat':'緯度', 'lon':'経度'},
    size_max=12, zoom=5, height=550)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.update_layout(template='plotly_dark')
if update_map:
    tw = get_twitter()
    show_and_save_plotly(fig, imgname, js=False, show=True, image=True, html=True)
    tw_body += " https://geneasyura.github.io/cov19-hm/longcovid.html "
    tweet_with_image(tw, "docs/images/{}".format(imgname), tw_body)
else:
    print("nothin to tweet about hokkaido cluster map.")

In [ ]:
def df_to_html(filename):
    with open(filename, "w", encoding='utf-8') as f:
        f.write('<style>\n')
        f.write('.MapsTblColNo { width:  50px; }\n')
        f.write('.MapsTblColSt { width:  80px; }\n')
        f.write('.MapsTblColNm { width: 270px; }\n')
        f.write('.MapsTblColOp { width: 270px; }\n')
        f.write('.MapsTblColEn { width:  80px; }\n')
        f.write('.MapsTblColOL { width: 100px; }\n')
        f.write('.MapsTblColRS { width:  80px; }\n')
        f.write('.MapsTblColCd { width: 100px; }\n')
        f.write('.MapsTblColHP { width:  80px; }\n')
        f.write('</style>\n')

        f.write('<table border="0">\n')
        f.write('  <thead style="display: block;" bgcolor="#000000">\n')
        f.write('    <tr style="text-align: left;">\n')
        f.write('      <th class="MapsTblColNo">八地方</th>\n')
        f.write('      <th class="MapsTblColSt">都道府県</th>\n')
        f.write('      <th class="MapsTblColNm">施設名称</th>\n')
        f.write('      <th class="MapsTblColOp">所在地</th>\n')
        f.write('      <th class="MapsTblColEn">診療可</th>\n')
        f.write('      <th class="MapsTblColOL">オンライン</th>\n')
        f.write('      <th class="MapsTblColRS">紹介状</th>\n')
        f.write('      <th class="MapsTblColCd">条件</th>\n')
        f.write('      <th class="MapsTblColHP">HP</th>\n')
        f.write('    </tr>\n')
        f.write('  </thead>\n')
        f.write('  <tbody style="display: block; overflow-x: hidden; overflow-y: scroll; height: 300px;">\n')
        for i, r in df.iterrows():
            f.write('<tr bgcolor="{}">'.format(["#606060", "#000000"][i % 2]))
            f.write('<td class="MapsTblColNo">{}</td>'.format(r['八地方区分']))
            f.write('<td class="MapsTblColSt">{}</td>'.format(r['都道府県']))
            f.write('<td class="MapsTblColNm">{}</td>'.format(r['施設名称']))
            f.write('<td class="MapsTblColOp">{}</td>'.format(r['所在地']))
            f.write('<td class="MapsTblColEn">{}</td>'.format(r['可']))
            f.write('<td class="MapsTblColOL">{}</td>'.format(r['オンライン']))
            f.write('<td class="MapsTblColRS">{}</td>'.format(r['紹介状']))
            f.write('<td class="MapsTblColCd">{}</td>'.format(r['条件']))
            f.write('<td class="MapsTblColHP">{}</td>'.format(r['HP']))
            f.write('</tr>\n')
        f.write('</tbody></table>\n')

In [ ]:
df_to_html('docs/_includes/longcovid-table.html')